![logo](images/untumbes.PNG)

<b><center>Prof. Dr. Jorge Zavaleta - zavaleta.jorge@gmail.com</center></b>

# Clasificación de datos
> La clasificación es el proceso de categorizar o agrupar datos en diferentes clases o categorías en función de determinadas características o atributos. 

## Importación de las bibliotecas

In [ ]:
# librarys
import numpy as np                          # calculo cientifico
import pandas as pd                         # manejo de dataframes
import matplotlib.pyplot as plt             # visualizacion grafica
import seaborn as sns                       # visualizacion grafica

## Lectura del dataset

In [ ]:
# paths
path_dataset ='data/'    # diretorio do dataset local
# dataset read
file_name='covid_19.csv'          # nome do dataset
dataset = pd.read_csv(path_dataset+file_name,sep=';',encoding='utf-8',low_memory=False) #index_col=0  #encoding='ISO-8859-1','latin-1')     # leitura do dataset
dataset.head()

In [ ]:
# shape
dataset.shape

## Seleccionar el dataset para clasificar

> Reduzir usando el objetivo (curados y obitos) - dataset['EVOLUCAO']

In [ ]:
# filtro para curados e obitos
curados_obitos = (dataset['EVOLUCAO']==1.0) | (dataset['EVOLUCAO']== 2.0)
# filtrar los curados y obitos en el dataset
data_cura_obito = dataset[curados_obitos]

In [ ]:
# nuevo dataset
data_cura_obito.head()

In [ ]:
# shape
data_cura_obito.shape

## Dataset de curados

In [ ]:
dataset_curados = data_cura_obito[data_cura_obito['EVOLUCAO']==1.0]
print('Curados:',dataset_curados.shape)

In [ ]:
# selec las caracteristicas: edad y sintomas
keys = dataset_curados.keys()
print(list(keys))

In [ ]:
# selecionr ls variables para el modelo
cols =['NU_IDADE_N','FEBRE','TOSSE','GARGANTA','DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 
       'DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','FATOR_RISC','PUERPERA', 'CARDIOPATI',
       'HEMATOLOGI','SIND_DOWN','HEPATICA','ASMA','DIABETES','NEUROLOGIC', 'PNEUMOPATI','IMUNODEPRE','RENAL',
       'OBESIDADE', 'OUT_MORBI','VACINA_COV','VACINA','UTI','EVOLUCAO']
data_curados = dataset_curados[cols]
data_curados.head(10)

In [ ]:
# shape
data_curados.shape

In [ ]:
#reset index para el index ser continuo
data_curados.reset_index(drop=True, inplace=True)
data_curados.head(10)

In [ ]:
# visualizando los ultimos registros del dataset
data_curados.tail(10)

In [ ]:
# criando copia
work_curados = data_curados.copy()

In [ ]:
# visualiza resltado
work_curados.head(10)

In [ ]:
# verifica dimensiones
work_curados.shape

>## Modifica valor 2.0 por 0.0

In [ ]:
# convert colunas en binario
list_keys = list(cols)
#
for k in range(1,len(list_keys)-1):
    col_name = list_keys[k]
    for i in range(0,167216):
        #print(i,col)
        v = work_curados.loc[i,col_name]
        if v == 1.0:
            pass
        elif v == 2.0:
            work_curados.loc[i,col_name]= 0.0
        else:
            pass

In [ ]:
# visualiza resultado
work_curados.head(10)

>## Eliminar las filas com 9.0

In [ ]:
#
# convert colunas en binario
list_keys = list(cols)
#
for i in range(0,167216):
    for k in range(1,len(list_keys)-1):
        col_name = list_keys[k]
        v9 = work_curados.loc[i,col_name]
        if v9 == 9.0:
            #print(i,col_name,v9)
            work_curados.drop(i, axis=0, inplace=True)
            break
        else:
            pass       

In [ ]:
# ver resultados
work_curados.head(10)

In [ ]:
# re-index
work_curados.reset_index(drop=True, inplace=True)
work_curados.head(10)

In [ ]:
# dimensions
dim = work_curados.shape
print(dim)
print('eliminados:',167216 - dim[0])

In [ ]:
# valores nulos - heatmap
import missingno as msno
%matplotlib inline
#msno.matrix(work_curados.sample(50)) # não funciona
msno.heatmap(work_curados)

In [ ]:
# 
def missing_values_dataset(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
curados = missing_values_dataset(work_curados)
curados

>## Llenar NaN com MEDIA??

In [ ]:
# convert colunas en binario
list_keys = list(cols)
#
for k in range(1,len(list_keys)-1):
    col_name = list_keys[k]
    #print('coluna:',col_name)
    media = work_curados[col_name].mean()
    #print('moda:',moda)
    if media >= 0.5:
        med = 1.0
    else:
        med = 0.0
    work_curados[col_name].fillna(med, inplace=True)
#
work_curados.head(10)

In [ ]:
#work_curados.drop('OUTRO_DES', axis=1, inplace=True)
#work_curados.head(10)

> ## Construyendo el dataset final

In [ ]:
#keys_col =  work_curados.keys()
new_cols = ['NU_IDADE_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP',
       'SATURACAO', 'VOMITO', 'FADIGA', 'OUTRO_SIN', 'FATOR_RISC', 'CARDIOPATI',
       'ASMA', 'DIABETES', 'OUT_MORBI','VACINA_COV', 'UTI', 'EVOLUCAO']
data_curados_final = work_curados[new_cols]
data_curados_final.head(10)

In [ ]:
data_curados_final.shape

## Dataset de Óbitos

In [ ]:
dataset_obitos = data_cura_obito[data_cura_obito['EVOLUCAO']==2.0]
print('Óbitos:',dataset_obitos.shape)

In [ ]:
obitos = dataset_obitos[new_cols]
obitos.head()

In [ ]:
obitos.shape

> ## EVOLUCION ==2.0 o 0?

In [ ]:
data_obitos = obitos.copy()
data_obitos.loc[data_obitos['EVOLUCAO'] == 2.0, 'EVOLUCAO'] = 0

In [ ]:
#
data_obitos.head(20)

>## Modifica 2.0 por 0

In [ ]:
# convert colunas en binario
# re-index
data_obitos.reset_index(drop=True, inplace=True)
cols = data_obitos.keys()
list_keys_o = list(cols)
#
for k in range(1,len(list_keys_o)-1):
    col_name = list_keys_o[k]
    for i in range(0,17884):
        #print(i,col_name)
        v = data_obitos.loc[i,col_name]
        if v == 1.0:
            pass
        elif v == 2.0:
            data_obitos.loc[i,col_name]= 0
        else:
            pass

In [ ]:
# visualiza resultado
data_obitos.head(10)

In [ ]:
data_obitos.shape

>## Elimina filas con valor 9

In [ ]:
#
# convert colunas en binario
list_keyso = list(new_cols)
#
for i in range(0,17884):
    for k in range(1,len(list_keyso)-1):
        col_name = list_keyso[k]
        v9 = data_obitos.loc[i,col_name]
        if v9 == 9.0:
            #print(i,col_name,v9)
            data_obitos.drop(i, axis=0, inplace=True)
            break
        else:
            pass     

In [ ]:
data_obitos.head(10)

In [ ]:
data_obitos.shape

> ## Llenar NaN con Media??

In [ ]:
# convert colunas en binario
list_keys_o = list(new_cols)
#
for k in range(1,len(list_keys_o)-1):
    col_name = list_keys_o[k]
    #print('coluna:',col_name)
    media = data_obitos[col_name].mean()
    #print('moda:',moda)
    if media >= 0.5:
        med = 1.0
    else:
        med = 0.0
    data_obitos[col_name].fillna(med, inplace=True)
#
data_obitos.head(10)

In [ ]:
# dimensions
data_obitos.shape

>## Verificando las dimensiones de los datasets CURADOS y OBITOS

In [ ]:
# dataset curados
data_curados_final.shape

In [ ]:
# dataset obitos
data_obitos.shape

In [ ]:
# Diferencia
c = data_curados_final.shape
o =data_obitos.shape
print('Diferencia:',c[0] - o[0])

>## Usando scikit-learn

In [ ]:
from sklearn.utils import resample
dfc = resample(data_curados_final, n_samples=16485, replace=False, stratify=data_curados_final,random_state=0)
dfc.head()

In [ ]:
dfc.shape

In [ ]:
# re-index
dfc.reset_index(drop=True, inplace=True)
dfc.head()

>## Unir dos dataset

In [ ]:
# Concatene os DataFrames verticalmente (sem modificar as colunas)
df_cura_obitos = pd.concat([dfc, data_obitos], ignore_index=True)
# re-index
df_cura_obitos.reset_index(drop=True, inplace=True)
df_cura_obitos.head()

In [ ]:
df_cura_obitos.tail(10)

In [ ]:
df_cura_obitos.shape

In [ ]:
#save dataset to csv
def save_dataset(dataset,path,dataset_name):
    dataset.to_csv(path+dataset_name,sep=';',encoding='utf-8',index=False)

In [ ]:
dataset = df_cura_obitos
path = 'data/'
dataset_name = 'curados_obitos_final.csv'
save_dataset(dataset,path,dataset_name)

---
<center><b>&copy; Jorge Zavaleta, 2024</b></center>